In [2]:
# Standard Python modules
import os, sys
import numpy as np
import pandas as pd
import xarray as xr
import datetime as dt

# plot styles/formatting
import seaborn as sns
import cmocean.cm as cmo
import cmocean

# matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import AxesGrid
from matplotlib.gridspec import GridSpec
from matplotlib.colorbar import Colorbar # different way to handle colorbar
import matplotlib.ticker as mticker
import matplotlib.patches as mpatches
import matplotlib.colors as mcolors
from mpl_toolkits.axes_grid1.inset_locator import mark_inset

# cartopy
import cartopy.crs as ccrs
from cartopy.mpl.geoaxes import GeoAxes
import cartopy.feature as cfeature

# extras
%matplotlib inline
import metpy.calc as mpcalc
from metpy.units import units
from scipy.ndimage import gaussian_filter    # smoothing contour lines

# import personal modules
# Path to modules
sys.path.append('../../modules')
import nclcmaps as nclc
from plotter import draw_basemap
from preprocess_dataframes import combine_ivt_ar_prec_df, df_annual_clim, calculate_ivt_prec_percentiles, df_AR_annual_clim
from plotter import draw_basemap, range_labels, calc_rose, build_rose

pd.options.display.float_format = "{:,.2f}".format # makes it so pandas tables display only first two decimals

In [ ]:
# Set up paths
server = "skyriver"
if server == "comet":
    path_to_data = '/cw3e/mead/projects/cwp140/scratch/dnash/data/'      # project data -- read only
elif server == "skyriver":
    path_to_data = '/home/dnash/comet_data/'

path_to_out  = '../out/'       # output files (numerical results, intermediate datafiles) -- read & write
path_to_figs = '../figs/'      # figures

In [ ]:
## load difference composites
fname = path_to_data + 'preprocessed/ERA5_ivt_250z_mslp_daily_diff_composite.nc'
ds_diff_final = xr.open_dataset(fname)
# convert geopotential to geopotential height (m)
ds_diff_final = ds_diff_final.assign(z=lambda ds_diff_final: ds_diff_final['z']/(9.80665))

## load pval composites
fname = path_to_data + 'preprocessed/ERA5_ivt_250z_mslp_daily_pval_composite.nc'
ds_pval_final = xr.open_dataset(fname)

In [ ]:
## load special composites
fname = path_to_data + 'preprocessed/ERA5/ERA5_ivt_250z_mslp_daily_composite_special.nc'
ds_special = xr.open_dataset(fname)
ds_special

len_special = [56, 1266, 566]

In [ ]:
## open precipitation and ivt dfs
## append precip to each community IVT df
option = 'a'
temporal_res = 'daily'
community_lst = ['Hoonah', 'Skagway', 'Klukwan', 'Yakutat', 'Craig', 'Kasaan']

df_lst = combine_ivt_ar_prec_df(option, temporal_res, community_lst) # combine dfs into list of dfs

## windrose precipitation bins - set to below 95th percentile and greater than or equal to 95th percentileperc = [0., .95, 1.]
perc = [0., 0.95, 1.]
perc_lbl = ['0%', '95%', '100%']
prec_bins_lst = []
for i, df in enumerate(df_lst):
    sublist = []
    sublist_ivt = []
    for j, (thres, lbl) in enumerate(zip(perc, perc_lbl)):
        sublist.append(df['prec'].describe(percentiles=[thres]).loc[[lbl]].values.tolist())
    flat_list = [item for l in sublist for item in l] # flatten the list
    flat_list.insert(0, 0)
    prec_bins_lst.append(flat_list)

perc_lbl.insert(0, 'clear')
prec_labels = range_labels(perc_lbl)

## windrose direction bins - set to every 30 degrees
## N, NNE, ENE, E, ESE, SE, SSE, S, SSW, SW, WSW, W, WNW, NW, and NNW
dir_bins = np.arange(-15, 375+30, 30)
dir_labels = (dir_bins[:-1] + dir_bins[1:]) / 2
rad_ticks = [10, 20, 30, 40, 45] # the ticks for the radial plot
print(prec_bins_lst)

In [ ]:
## Open GMTED2010 7.5 arc second elevation data
elev = xr.open_dataset(path_to_data + 'preprocessed/seak_gmted_mea075.nc')

In [ ]:
xs = [-135.4519, -135.3277, -135.8894, -139.671, -133.1358, -132.4009]
ys = [58.1122, 59.4538, 59.3988, 59.5121, 55.4769, 55.5400]
lbl1 = ['Hoonah', 'Skagway', 'Klukwan', 'Yakutat', 'Craig', 'Kasaan']
community_lst = lbl1
lbl_align = ['center', 'left', 'right', 'center', 'right', 'left'] # where the labels go
ext1 = [-141., -130., 54., 61.] # extent of SEAK

# list of extents for each community
dx = 1.13 # this works for our paper
dy = 1.13 # this works for our paper

ext_lst = []
for x,y in zip(xs, ys):
    ext_lst.append([x-dx, x+dx, y-dy, y+dy])

temporal_res = 'daily'
fname = path_to_data + 'preprocessed/SEAK-WRF_PCPT_{0}_composite.nc'.format(temporal_res)
prec_comp  = xr.open_dataset(fname)
## calculate average of all 6 communities
prec_mean = prec_comp.mean('community')

fname = path_to_data + 'preprocessed/SEAK-WRF_UV_{0}_composite.nc'.format(temporal_res)
uv_comp  = xr.open_dataset(fname)

## calculate average of all 6 communities
uv_mean = uv_comp.mean('community')

## ivt composites
fname = path_to_data + '/preprocessed/ERA5/ERA5_ivt_daily.nc'
ivt_comp = xr.open_dataset(fname)
ivt_comp

In [ ]:
# Create figure
fig = plt.figure(figsize=(11, 10.5))
fig.dpi = 300
fname = path_to_figs + 'summary_fig'
fmt = 'png'

nrows = 4
ncols = 2

## Use gridspec to set up a plot with a series of subplots that is
## n-rows by n-columns
gs = GridSpec(nrows, ncols, height_ratios=[1, 0.05, 1, 0.05], width_ratios = [1, 1], wspace=0.2, hspace=0.15)
## use gs[rows index, columns index] to access grids

#######################
### IVT CLIMATOLOGY ###
#######################


######################
### IVT DIFFERENCE ###
######################


########################
### PRECIP COMPOSITE ###
########################

####################
### IVT WINDROSE ###
####################
